<a href="https://colab.research.google.com/github/shrigoa/random/blob/main/Random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem:
1. Manage internet bandwidth usage on a ship.

Important data:
1. Access to two communication links each contracted with 15TB of bandwidth for a month, thus, total of 30TB.

Goals:
1. Optimal usage: Bandwidth should be consumed only when required and minimized when not needed.
2.a. Fairness Across Time: Quality of Experience (QoE) should be consistent across different timeslots.
2.b. Significant variations — such as excellent QoE in some periods and poor QoE in others — must be inimized to avoid service quality alarms.
2.c. Demand met percentage is the QoE. It should not vary much.

Given Data:
1. Per hour demand data for upstream and downstream traffic
- Variations of 20-25% are possible
- Columns: Time slots, downstream demand, upstream demand
2. Ship arrival at ports
- Variation of 1-2 hours earlier or later.
- Demand drops 50% at port compared to sea.
- Columns: Port, Arrival Time, Departure Time
3. Link Downtime
- One of the links will be unavailable on certain days.
- This downtime must be configurable in the code.
- During downtime, the algorithm must reserve enough bandwidth on the remaining active link to maintain stable QoE and avoid service degradation.
4. Consumption Limits:
- Each link supports maximum bandwidth of 500Mbps at any given point.
- Bandwidth planning is done in 5 min time-slots.
- Bandwidth volume (e.g., 1 GB in 5 minutes) will internally be converted into rate for enforcement.
5. End of the month requirement:
- Allocated 30TB should be completely used. (CONSTRAINT)
- No under/over usage allowed.
6. Objective:
- Demand met percentage is the QoE (OBJECTIVE FUNCTION).
- Needs to be as high as possible.

7. Parameters:
- Number of 5 mins slots per day: 288 slots
- Number of slots per hour: 12 slots
- Number of days: 30 days
- Seconds in each slot:
- Downtime start day: 19
- Downtime end day: 27
- Capacity per link: 15 TB
- Number of links: 2
- Number of links up during downtime: 1
- Maximum bandwidth of any link: 500 Mbps

7. Decision Variables
- How much data is consumed in each 5 mins slot: data_consumed[day, hour, slot]

8. Objective Function
- demand[day, hour] - sum of data_consumed[day, hour, slot] over all slots for each day, for each hour data







In [1]:
#Import packages
import numpy as np
import pandas as pd
import math
import gurobipy as gp
from gurobipy import GRB

In [2]:
#Parameters
#Fixed Parameters
hours_in_day = 24
mins_in_hour = 60
secs_in_min = 60

#Configurable Parameters
slot_mins = 5
num_planning_days = 30
active_links = 2
cap_per_link = 15000000
max_bandwidth_per_link = 500
downtime_start_day = 19
downtime_end_day = 27
active_links_during_downtime = 1

#Derived Parameters
seconds_in_slot = slot_mins * secs_in_min
seconds_in_hour = mins_in_hour * secs_in_min
slots_per_hour = mins_in_hour/slot_mins
slots_per_day = int(slots_per_hour * hours_in_day)
max_data_per_slot_per_link = max_bandwidth_per_link * seconds_in_slot
max_data_per_slot = max_data_per_slot_per_link * active_links
max_data_per_hour = max_data_per_slot * slots_per_hour

In [3]:
#Define Model
env = gp.Env()
env.setParam('OutputFlag', 0)
mip = gp.Model('PlanningMIP', env = env)

Restricted license - for non-production use only - expires 2026-11-23


In [4]:
#Decision Variables
slot_consumption = {}
hour_consumption = {}

for day in range(num_planning_days):
  for hour in range(hours_in_day):
    hour_consumption[day, hour] = mip.addVar(lb = 0, ub = max_data_per_hour, vtype=GRB.CONTINUOUS, name = f'hour_consumption_{day}_{hour}')
    # for slot in range(slots_per_day):
    #   slot_consumption[day, hour, slot] = mip.addVar(lb = 0, ub = max_data_per_slot, vtype=GRB.CONTINUOUS, name = f'slot_consumption_{day}_{hour}_{slot}')

In [5]:
#Constraints

In [6]:
#Objective
data_consumed_exp = gp.LinExpr()
for day in range(num_planning_days):
  for hour in range(hours_in_day):
    data_consumed_exp += hour_consumption[day, hour]
mip.setObjective(data_consumed_exp, GRB.MAXIMIZE)

In [7]:
#Solve Model
mip.write('mip.lp')
mip.optimize()

In [8]:
#Result
if mip.status == GRB.OPTIMAL:
  print("Found Optimal Solution!!")
  print("Obj", mip.ObjVal)
  for day in range(num_planning_days):
    for hour in range(hours_in_day):
      if hour_consumption[day, hour].X > 0:
        print()
        #print(day, hour, hour_consumption[day, hour].X)

Found Optimal Solution!!
0 0 3600000.0
0 1 3600000.0
0 2 3600000.0
0 3 3600000.0
0 4 3600000.0
0 5 3600000.0
0 6 3600000.0
0 7 3600000.0
0 8 3600000.0
0 9 3600000.0
0 10 3600000.0
0 11 3600000.0
0 12 3600000.0
0 13 3600000.0
0 14 3600000.0
0 15 3600000.0
0 16 3600000.0
0 17 3600000.0
0 18 3600000.0
0 19 3600000.0
0 20 3600000.0
0 21 3600000.0
0 22 3600000.0
0 23 3600000.0
1 0 3600000.0
1 1 3600000.0
1 2 3600000.0
1 3 3600000.0
1 4 3600000.0
1 5 3600000.0
1 6 3600000.0
1 7 3600000.0
1 8 3600000.0
1 9 3600000.0
1 10 3600000.0
1 11 3600000.0
1 12 3600000.0
1 13 3600000.0
1 14 3600000.0
1 15 3600000.0
1 16 3600000.0
1 17 3600000.0
1 18 3600000.0
1 19 3600000.0
1 20 3600000.0
1 21 3600000.0
1 22 3600000.0
1 23 3600000.0
2 0 3600000.0
2 1 3600000.0
2 2 3600000.0
2 3 3600000.0
2 4 3600000.0
2 5 3600000.0
2 6 3600000.0
2 7 3600000.0
2 8 3600000.0
2 9 3600000.0
2 10 3600000.0
2 11 3600000.0
2 12 3600000.0
2 13 3600000.0
2 14 3600000.0
2 15 3600000.0
2 16 3600000.0
2 17 3600000.0
2 18 3600000.0
